In [101]:
from keras.preprocessing import image
import numpy as np
import os
from os import listdir
from os.path import isfile, join
from skimage.io import imread
from skimage.transform import resize

IMG_SIZE = 128

path = 'data'
classes = next(os.walk(path))[1]
images = []
labels = []
for label, class_name in enumerate(classes):
    dirs = next(os.walk(join(path, class_name)))[1]
    for dir_name in dirs:
        files = next(os.walk(join(path, class_name, dir_name)))[2]
        for file in files:
            filename = join(path, class_name, dir_name, file)
            try:
                #img = imread(filename)[:, :, :3] # skip alpha channel
                #img = resize(img, (IMG_SIZE, IMG_SIZE), mode='constant', preserve_range=True)
                img = image.load_img(filename, target_size=(IMG_SIZE, IMG_SIZE))
                img_arr = image.img_to_array(img)
            except:
                pass
            
            if not img_arr.shape == (IMG_SIZE, IMG_SIZE, 3):
                print("Unexpected shape")
                continue
                
            images.append(img_arr)
            labels.append(label)    

print("Loaded %d images" % len(images))

/Users/wouter/anaconda2/envs/deep-matting/lib/python3.6/site-packages/PIL/Image.py:916: UserWarning: Palette images with Transparency   expressed in bytes should be converted to RGBA images
  'to RGBA images')


Loaded 1217 images


In [102]:
import random
#from keras.applications.vgg16 import preprocess_input
from keras.applications.resnet50 import preprocess_input

# shuffle all samples
samples = list(zip(images,labels))
random.shuffle(samples)
x,y = zip(*samples)

# make numpy arrays
x = np.array(x)
y = np.array(y)

# one hot encoding of labels
y = keras.utils.to_categorical(y)

# perform preprocessing for vgg16
x = preprocess_input(x)

# split test / train set
split = 100
train_size = 500
test_x = x[:split]
test_y = y[:split]
train_x = x[split:split + train_size]
train_y = y[split:split + train_size]

In [95]:
import matplotlib.pyplot as plt

for i in range(0,10):
    print(test_y[i])
    plt.imshow(test_x[i])
    plt.show()

[1. 0.]


ValueError: Floating point image RGB values must be in the 0..1 range.

[0. 1.]


ValueError: Floating point image RGB values must be in the 0..1 range.

[0. 1.]


...

ValueError: Floating point image RGB values must be in the 0..1 range.

[0. 1.]


ValueError: Floating point image RGB values must be in the 0..1 range.

[1. 0.]


ValueError: Floating point image RGB values must be in the 0..1 range.

In [108]:
import keras
from keras import applications
from keras import backend as K
from keras.models import Sequential, Model
from keras import layers
from keras.layers import GlobalMaxPooling2D, GlobalAveragePooling2D, Dense, Activation, Reshape, Flatten, BatchNormalization, AtrousConvolution2D, Conv2D, MaxPooling2D, Dropout, Input, merge
from keras.optimizers import Adam
from keras.layers.convolutional import Deconvolution2D, Conv2DTranspose
from keras.callbacks import ModelCheckpoint

base_model = keras.applications.VGG16(include_top=False, weights='imagenet', input_shape = (IMG_SIZE, IMG_SIZE, 3))
#base_model = keras.applications.ResNet50(weights='imagenet', include_top=False, input_shape = (IMG_SIZE, IMG_SIZE, 3))
base_model.trainable = False

model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(256, activation='relu'),
    Dropout(0.3),
    Dense(2, activation='softmax')
])

model.summary()


model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
#model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.001), metrics=['accuracy'])

history = model.fit(x = train_x, y = train_y, batch_size = 8, validation_split = 0.1, epochs = 10, verbose=1)

model.evaluate(test_x, test_y)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 4, 4, 512)         14714688  
_________________________________________________________________
global_average_pooling2d_4 ( (None, 512)               0         
_________________________________________________________________
dense_43 (Dense)             (None, 256)               131328    
_________________________________________________________________
dropout_7 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_44 (Dense)             (None, 2)                 514       
Total params: 14,846,530
Trainable params: 131,842
Non-trainable params: 14,714,688
_________________________________________________________________
Train on 450 samples, validate on 50 samples
Epoch 1/10
450/450 [==============================] - 70s 156ms/step - loss: 1.

KeyboardInterrupt: 

In [109]:
model.evaluate(test_x, test_y)

100/100 [==============================] - 14s 139ms/step


[0.23701439529677373, 0.96]

In [69]:
y_hat = model.predict(test_x)
print(y_hat)

[[0.38908738 0.6109127 ]
 [0.47373715 0.5262628 ]
 [0.40086794 0.5991321 ]
 [0.46913293 0.5308671 ]
 [0.38316596 0.61683404]
 [0.4267387  0.57326126]
 [0.47674027 0.52325976]
 [0.45874903 0.541251  ]
 [0.427541   0.572459  ]
 [0.44855446 0.5514456 ]
 [0.3971409  0.60285914]
 [0.42406145 0.5759386 ]
 [0.44475955 0.55524045]
 [0.42985123 0.57014877]
 [0.34305742 0.6569426 ]
 [0.4302698  0.5697303 ]
 [0.43315592 0.5668441 ]
 [0.48240387 0.5175961 ]
 [0.4594394  0.54056054]
 [0.38394773 0.6160523 ]
 [0.40269375 0.5973062 ]
 [0.45673403 0.543266  ]
 [0.46743104 0.53256893]
 [0.40383372 0.59616625]
 [0.38917166 0.61082834]
 [0.4740546  0.52594537]
 [0.49647093 0.503529  ]
 [0.43913248 0.5608675 ]
 [0.41374683 0.58625317]
 [0.42454112 0.5754589 ]
 [0.43222925 0.5677708 ]
 [0.4200079  0.57999206]
 [0.45599285 0.5440071 ]
 [0.45551038 0.5444897 ]
 [0.41849178 0.5815082 ]
 [0.41860554 0.58139443]
 [0.41726547 0.5827345 ]
 [0.43595937 0.56404066]
 [0.40336227 0.5966377 ]
 [0.3767873  0.6232127 ]
